In [2]:
!pip install webdriver-manager
!pip install pandas requests beautifulsoup4 yfinance tqdm




[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
# Thêm thư viện cho Chờ đợi Rõ ràng (Explicit Wait)
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
import random
import time 
import mysql.connector
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from tqdm import tqdm
import numpy as np
from time import sleep
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException


In [9]:
## HAM 1: CRAWL

API_URL = 'https://inav.ice.com/api/1/tse/iopv/table?type=etf&language=en'
HEADERS = {'User-Agent': 'Mozilla/5.0', 'Referer': 'https://inav.ice.com'}

def get_etf_codes():
    """Lấy danh sách các ETF code theo đúng thứ tự API trả về."""
    try:
        r = requests.get(API_URL, headers=HEADERS, timeout=10)
        r.raise_for_status()
        rows = r.json().get("rows", [])
        # dùng list để giữ thứ tự, và loại code rỗng (nếu có)
        return [row["code"] for row in rows if row.get("code")]
    except requests.RequestException as e:
        print(f"⚠️ Lỗi khi lấy ETF codes từ API: {e}")
        return []

codes = get_etf_codes()
df = pd.DataFrame(codes, columns=["code"])

# 1. Khởi tạo WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--log-level=3")
driver = webdriver.Chrome(options=chrome_options)

# 2. Chuẩn bị dữ liệu và tham số
WAIT_TIMEOUT = 30 
data_list_equity = []
# Lấy TẤT CẢ các mã từ cột 'code' của DataFrame df
codes_to_process = df['code'].tolist()

print("\n--- Bắt đầu thu thập các chỉ số định giá (PE, PB, PS, PC) cho TẤT CẢ 386 mã ---")
print(f"Tổng số mã cần xử lý: {len(codes_to_process)}")

# 3. Lặp qua TẤT CẢ 386 mã trong df
for index, base_code in enumerate(codes_to_process):
    # Đảm bảo base_code là chuỗi
    base_code = str(base_code)
    full_code = f"{base_code}.T"
    url = f"https://finance.yahoo.com/quote/{full_code}/holdings/"

    print(f"[{index + 1}/{len(codes_to_process)}] Đang xử lý mã: {full_code}")

    # Khởi tạo dict và đặt giá trị mặc định là None/NA cho các chỉ số
    etf_data = {
        'etf_code': base_code,
        'price/earnings': None,
        'price/book': None,
        'price/sales': None,
        'price/cashflow': None
    }

    try:
        driver.get(url)

        # Chờ đợi tường minh cho bảng Equity Holdings xuất hiện
        WebDriverWait(driver, WAIT_TIMEOUT).until(
            EC.presence_of_element_located((By.XPATH, "//section[@data-testid='equity-holdings']//table"))
        )

        # Tìm tbody của bảng Equity Holdings
        equity_tbody = driver.find_element(
            By.XPATH,
            "//section[@data-testid='equity-holdings']//table/tbody"
        )

        # Lấy các hàng (<tr>)
        rows = equity_tbody.find_elements(By.TAG_NAME, "tr")
        crawled_count = 0

        for row in rows:
            try:
                # Cột 1: Tên chỉ số | Cột 2: Giá trị
                metric_name = row.find_element(By.XPATH, "./td[1]").text
                metric_value = row.find_element(By.XPATH, "./td[2]").text

                if metric_name == 'Price/Earnings':
                    etf_data['price/earnings'] = metric_value
                    crawled_count += 1
                elif metric_name == 'Price/Book':
                    etf_data['price/book'] = metric_value
                    crawled_count += 1
                elif metric_name == 'Price/Sales':
                    etf_data['price/sales'] = metric_value
                    crawled_count += 1
                elif metric_name == 'Price/Cashflow':
                    etf_data['price/cashflow'] = metric_value
                    crawled_count += 1

                # Nếu đã lấy đủ 4 chỉ số thì thoát
                if crawled_count == 4:
                    break

            except NoSuchElementException:
                continue

        data_list_equity.append(etf_data)

        if crawled_count < 4:
            print(f"⚠️ Cảnh báo: Chỉ lấy được {crawled_count}/4 chỉ số cho {full_code}.")
        else:
            print(f"✅ Đã lấy đủ 4 chỉ số.")

    except TimeoutException:
        print(f"🔴 Thất bại: Hết thời gian chờ ({WAIT_TIMEOUT}s) cho {full_code}. Ghi nhận NA.")
        # Thêm mã thất bại với giá trị NA
        data_list_equity.append({
            'etf_code': base_code, 'price/earnings': pd.NA, 'price/book': pd.NA,
            'price/sales': pd.NA, 'price/cashflow': pd.NA
        })
    except Exception as e:
        print(f"🔴 Lỗi không xác định khi xử lý code {full_code}: {e}. Ghi nhận NA.")
        # Thêm mã thất bại với giá trị NA
        data_list_equity.append({
            'etf_code': base_code, 'price/earnings': pd.NA, 'price/book': pd.NA,
            'price/sales': pd.NA, 'price/cashflow': pd.NA
        })

    # Giãn cách giữa các request
    time.sleep(1.5)

# 4. Đóng WebDriver
driver.quit()

# 5. Tạo DataFrame kết quả
df_equity_result = pd.DataFrame(data_list_equity)

# 6. Sắp xếp lại DataFrame theo thứ tự gốc của df
## Tạo cột thứ tự (ranking) dựa trên DataFrame df gốc
code_order = df['code'].tolist()
code_map = {str(code): i for i, code in enumerate(code_order)}

## Ánh xạ etf_code sang thứ tự
df_equity_result['order'] = df_equity_result['etf_code'].astype(str).map(code_map)

## Sắp xếp DataFrame theo thứ tự mã gốc
df_equity_final = df_equity_result.sort_values(by=['order']).reset_index(drop=True)

## Xóa cột 'order'
df_equity_final = df_equity_final.drop(columns=['order'])

# 7. In kết quả cuối cùng
print("\n--- Kết Quả DataFrame Cuối Cùng (df_equity_final) ---")
print(f"Tổng số hàng dữ liệu cuối cùng: {len(df_equity_final)}")
print(df_equity_final.head())

In [22]:
## HAM 2: XU LI

for col in df_final_updated_data.columns:
    if col != 'etf_code':
        df_equity_final[col] = pd.to_numeric(df_equity_final[col], errors='coerce')

df_final_updated_data.rename(columns={'price/earnings' : 'price_earnings','price/book':'price_book','price/sales':'price_sales',
                                      'price/cashflow':'price_cashflow'}, inplace=True)


In [26]:
## HAM 3: UPSERT MYSQL
def upsert_to_mysql(df: pd.DataFrame, table_name: str = 'etf_equity_holdings_train'):
    """
    Thực hiện kết nối và UPSERT (Update or Insert) dữ liệu từ DataFrame vào MySQL.
    Sử dụng 'etf_code' làm khóa DUY NHẤT để kiểm tra trùng lặp.
    """
    try:
        # 1. Kết nối MySQL 
        cnx = mysql.connector.connect(
            user='root',
            password='Phunghainam2004!',
            host='127.0.0.1',
            database='train'
        )
        cursor = cnx.cursor()

        # 2. Định nghĩa các cột
        
        cols_to_use = [
            'etf_code', 'price_earnings', 'price_book', 
            'price_sales', 'price_cashflow']
        
        # 3. Tạo cú pháp SQL cho UPSERT
        col_str = ", ".join([f"`{c}`" for c in cols_to_use])
        placeholder_str = ", ".join(["%s"] * len(cols_to_use))
        
        # Tạo chuỗi cho phần ON DUPLICATE KEY UPDATE
        
        update_str = ", ".join([f"`{c}` = VALUES(`{c}`)" for c in cols_to_use])

        sql = f"""
        INSERT INTO `{table_name}` ({col_str})
        VALUES ({placeholder_str})
        ON DUPLICATE KEY UPDATE
        {update_str}
        """

        # 4. Chuẩn bị dữ liệu và thực thi
        df_to_insert = df[cols_to_use]
        data_to_insert = [
            tuple(None if pd.isna(x) else x for x in row) 
            for row in df_to_insert.itertuples(index=False, name=None)
        ]
        
        # Sử dụng executemany để chèn nhiều dòng cùng lúc (tối ưu hơn loop)
        cursor.executemany(sql, data_to_insert)
        
        # 5. Commit và Đóng kết nối
        cnx.commit()
        print(f"✅ Đã UPSERT {cursor.rowcount} dòng vào bảng `{table_name}` thành công!")

    except mysql.connector.Error as err:
        print(f"❌ Lỗi MySQL: {err}")
    
    finally:
        if 'cursor' in locals() and cursor:
            cursor.close()
        if 'cnx' in locals() and cnx and cnx.is_connected():
            cnx.close()



df_for_upsert = df_final_updated_data

upsert_to_mysql(df_for_upsert)

✅ Đã UPSERT 0 dòng vào bảng `etf_equity_holdings_train` thành công!
